<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/Shortest_Paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from collections import defaultdict
from math import log, e
import csv
import pandas as pd
import numpy as np
from typing import List
from sys import maxsize
from collections import deque
import random

In [29]:
# Print out the 'list' to the 'name.csv' file 

def printOut(name, list):
    with open(name + '.csv', 'w', newline='') as csv_1:
        csv_out = csv.writer(csv_1)
        csv_out.writerows([list[index]] for index in range(0, len(list)))

In [30]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [31]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [32]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
# Entropy by numpy library

def entropy(labels, base=None):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    value, counts = np.unique(labels, return_counts=True)
    probs = counts/n_labels
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.
    
    base = e if base is None else base
    for i in probs:
        ent -= i * log(i, base)
        
    return ent

In [34]:
paths = []

for i in range(len(connections)):
  src = i
  dest = 0
  route = []
  for j in connections[i]:
    if j == 1 and src < dest:
      route.append([src,dest])
    dest += 1
  paths.append(route)

In [35]:
# Function to form edge between
# two vertices src and dest
 
def add_edge(adj: List[List[int]],
             src: int, dest: int) -> None:
    adj[src].append(dest)
    adj[dest].append(src)

# Function which finds all the paths
# and stores it in paths array
def find_paths(paths: List[List[int]], path: List[int],
               parent: List[List[int]], n: int, u: int) -> None:
    # Base Case
    if (u == -1):
        paths.append(path.copy())
        return
 
    # Loop for all the parents
    # of the given vertex
    for par in parent[u]:
 
        # Insert the current
        # vertex in path
        path.append(u)
 
        # Recursive call for its parent
        find_paths(paths, path, parent, n, par)
 
        # Remove the current vertex
        path.pop()

# Function which performs bfs
# from the given souce vertex
def bfs(adj: List[List[int]],
        parent: List[List[int]], n: int,
        start: int) -> None:
 
    # dist will contain shortest distance
    # from start to every other vertex
    dist = [maxsize for _ in range(n)]
    q = deque()
 
    # Insert source vertex in queue and make
    # its parent -1 and distance 0
    q.append(start)
    parent[start] = [-1]
    dist[start] = 0
 
    # Untill Queue is empty
    while q:
        u = q[0]
        q.popleft()
        for v in adj[u]:
            if (dist[v] > dist[u] + 1):
 
                # A shorter distance is found
                # So erase all the previous parents
                # and insert new parent u in parent[v]
                dist[v] = dist[u] + 1
                q.append(v)
                parent[v].clear()
                parent[v].append(u)
 
            elif (dist[v] == dist[u] + 1):
 
                # Another candidate parent for
                # shortes path found
                parent[v].append(u)
 
# Function which prints all the paths
# from start to end
def getPaths(adj: List[List[int]], n: int,
                start: int, end: int) -> None:
    revPaths = []
    path = []
    parent = [[] for _ in range(n)]

    paths = []
 
    # Function call to bfs
    bfs(adj, parent, n, start)
 
    # Function call to find_paths
    find_paths(revPaths, path, parent, n, end)
    for v in revPaths:
        v = v[::-1]
        paths.append(v)

    return paths

In [36]:
# Number of vertices
n = 83

# Number of pairs
pairs = 0
for i in paths:
  for j in i:
    pairs+=1

# array of vectors is used
# to store the graph
# in the form of an adjacency list
adj = [[] for i in range(len(paths))]

for i in paths:
  for j in i:
    add_edge(adj, j[0], j[1])

In [37]:
shortest_paths = []

for i in range(len(paths)):
  for j in range(len(paths)):
    if i!=j:
      shortest_paths.append(getPaths(adj, n, i, j))

In [38]:
routingTable = []

for j in range(100):
  routes = []
  for i in shortest_paths:
    routes.append(random.choice(i))
  routingTable.append(routes)

In [44]:
'''entropyTables = []

for i in routingTable:
  temp = []
  for j in i:
    temp.append(entropy(j))
  entropyTables.append(temp)'''

'entropyTables = []\n\nfor i in routingTable:\n  temp = []\n  for j in i:\n    temp.append(entropy(j))\n  entropyTables.append(temp)'

In [50]:
epochSize = len(connections[0])
ent = []

cnt = 0
for i in entropyTables:
  temp = []
  table = []
  for j in i:
    if cnt == epochSize:
      table.append(temp)
      temp = []
      cnt = 0
    if cnt < epochSize:
      temp.append(j)
      cnt += 1
  cnt = 0
  ent.append(table)

In [52]:
ent